In [2]:
import geopandas as gpd
import networkx as nx
from shapely.geometry import LineString
from shapely.ops import snap

gdf = gpd.read_file("../data/UKR_networks.gpkg", layer="UKR_networks")

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
4370,242943907,5115,tertiary,None,О-102408,B,0,0,F,F,"LINESTRING (29.83948 49.77703, 29.83925 49.777..."


In [28]:
from shapely.ops import unary_union, linemerge, split
from shapely.geometry import MultiLineString, LineString
import momepy

roads = gdf.copy()
# Merge all roads into a single MultiLineString
merged = unary_union(roads.geometry)
x = merged.segmentize(max_segment_length=1000)
roads_multi = gpd.GeoDataFrame(data={"id":[1], "geometry":[x]})
gdf_roads = roads_multi.explode()
G = momepy.gdf_to_nx(gdf_roads, approach="primal")
nodes, edges, sw = momepy.nx_to_gdf(G, points=True, lines=True, spatial_weights=True)
nodes.to_file("../data/UKR_networks.gpkg", layer="nodes_fix")

C:\Users\dkerr\AppData\Local\Temp\ipykernel_25104\1227152626.py:10: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  gdf_roads = roads_multi.explode()
C:\Users\dkerr\Miniconda3\envs\gisrede\lib\site-packages\libpysal\weights\weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 8 disconnected components.
  warnings.warn(message)


In [41]:
edges

,id,geometry,mm_len,node_start,node_end
0,1,"LINESTRING (30.52383 50.46943, 30.52389 50.469...",0.002288,0,1
1,1,"LINESTRING (30.52341 50.46973, 30.52364 50.469...",0.000518,0,183
2,1,"LINESTRING (30.52459 50.46732, 30.52462 50.46713)",0.000193,1,7099
3,1,"LINESTRING (30.64523 50.41266, 30.64531 50.412...",0.003775,2,3
4,1,"LINESTRING (30.64523 50.41266, 30.64540 50.412...",0.000935,2,9488
...,...,...,...,...,...
13801,1,"LINESTRING (30.82951 50.51527, 30.83023 50.51569)",0.000832,11398,11399
13802,1,"LINESTRING (30.83023 50.51569, 30.83049 50.51584)",0.000299,11399,11400
13803,1,"LINESTRING (30.83049 50.51584, 30.83073 50.51598)",0.000273,11400,11401
13804,1,"LINESTRING (30.83073 50.51598, 30.83144 50.516...",0.002136,11401,11402


In [40]:
import pandas as pd
G = nx.MultiGraph()
G.add_nodes_from(nodes.nodeID.unique().tolist())
for index, row in edges.iterrows():
    G.add_edge(row.node_start, row.node_end, weight=row.mm_len)

shortest_path = nx.shortest_path(G, 10320, 7656, weight="weight")
edges_in_path = []
for i in range(len(shortest_path) - 1):
    start_node = shortest_path[i]
    end_node = shortest_path[i + 1]
    edge = edges[
        (edges['node_start'] == start_node) & (edges['node_end'] == end_node) |
        (edges['node_end'] == start_node) & (edges['node_start'] == end_node)
    ]
    edges_in_path.append(edge)
gdf_route = gpd.GeoDataFrame(pd.concat(edges_in_path))
gdf_route.to_file("../data/UKR_networks.gpkg", layer="route_test_fix", driver="GPKG")